<a href="https://colab.research.google.com/github/mcash8/IEEEAeroConf25/blob/main/Code/LossFunction/AbileneNaiveSlnLoss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gurobipy  # install gurobipy, if not already installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 30.3 MB/s eta 0:00:00


In [3]:
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

### Utility Functions


In [4]:
'''
    Utility Functions
'''

import pandas as pd
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
import gurobipy as gb
import random
import itertools as it


'''
    Build a fully connected network
'''

def fully_connected_network(num_nodes = 4, capacity = 100):

     # Create a directed graph
    G = nx.DiGraph()

    # Add nodes
    G.add_nodes_from(range(num_nodes))

    # Add edges with capacity
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                G.add_edge(i, j, capacity=capacity)

    return G


'''
    Build a star network
'''

def star_network(num_nodes = 4, capacity = 100):
    if num_nodes < 2:
        raise ValueError("The number of nodes must be at least 2 to create a star network.")

    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes
    G.add_nodes_from(range(num_nodes))

    # The central node is node 0
    central_node = 0

    # Add edges from the central node to all other nodes
    for node in range(1, num_nodes):
        G.add_edge(central_node, node, capacity=capacity)
        G.add_edge(node, central_node, capacity=capacity)

    return G

'''
    Build a line network
'''

def line_network(num_nodes = 4, capacity = 100):

    # Create an undirected graph
    G = nx.DiGraph()

    # Add nodes
    G.add_nodes_from(range(num_nodes))

    # Add edges with specified capacity
    for i in range(num_nodes - 1):
        # Connect node i to node i+1
        G.add_edge(i, i + 1, capacity=capacity)
        G.add_edge(i+1, i, capacity=capacity)

    return G

'''
    Cosine Similarity Function
'''

def cosine_similarity(A, B):

    # Flatten the matrices
    vec1 = A.flatten()
    vec2 = B.flatten()

    # Compute dot product
    dot_product = np.dot(vec1, vec2)

    # Compute norms
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)

    # Compute cosine similarity
    similarity = dot_product / (norm1 * norm2)

    return 1-similarity

'''
    Build the Abilene Topology
'''
def abilene_topo():
    G = nx.DiGraph()

    G.add_nodes_from(range(12))

    #add edges
    with open('data/topo.txt', 'r') as f:
        for line in f:
            parts = line.strip().split()

            # Extract node1, node2, and capacity
            node1 = int(parts[0])
            node2 = int(parts[1])
            capacity = float(parts[2])

            # Add the edges with the specified capacity
            G.add_edge(node1, node2, capacity=capacity)
            G.add_edge(node2, node1, capacity=capacity)

    return G

def abilene_topo_lossfn():
        '''
            If we use the original topology the MLU will be very small and differentiating is difficult.
            This topology scales the capacity values to be between 1 and 0.25.
        '''
        G = nx.DiGraph()

        G.add_nodes_from(range(12))

        #add edges
        with open('topo.txt', 'r') as f:
            for line in f:
                parts = line.strip().split()

                # Extract node1, node2, and capacity
                node1 = int(parts[0])
                node2 = int(parts[1])

                if (int(parts[2]) == 9920000):
                    capacity = 10
                elif (int(parts[2]) == 2480000):
                    capacity = 2

                # Add the edges with the specified capacity
                G.add_edge(node1, node2, capacity=capacity)
                G.add_edge(node2, node1, capacity=capacity)

        return G


'''
    Functions for MCF
'''

def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

def  getCosts(G):
    '''
        Get cost for edges in G
    '''
    has_cost_attribute = all('cost' in G.edges[edge] for edge in G.edges())

    if has_cost_attribute:
        cost = nx.get_edge_attributes(G, 'cost')
    else:
        # If costs aren't specified, make uniform.
        nx.set_edge_attributes(G, name='cost', values=1)
        cost = nx.get_edge_attributes(G, 'cost')

    return cost

def getCapacity(G):
    '''
        Get capacity of edges
    '''
    return nx.get_edge_attributes(G, 'capacity')

def getLoadandFlowVars(m, V, arcs, cost):

    f = m.addVars(V, V, arcs, obj=cost, name='flow')
    l = m.addVars(arcs, lb=0.0, name='tot_traf_across_link')

    return f, l

def getLinkUtilization(m, l, f, arcs):

    '''Link util = sum over flows for each od pair '''

    # Link utilization = sum of flow
    m.addConstrs((l[i, j] == f.sum('*', '*', i, j) for i, j in arcs), 'l_sum_traf',)

    return

def getCapacityConstraint(m, l, capacity, arcs):
    '''Link utilzation can not exceed link capacity'''
    # Add capacity constraints
    m.addConstrs(
        (l[i, j] <= capacity[i,j] for i, j in arcs),
        'traf_below_cap',
        )

    return

def getFlowConservationConstraint(m, D, V, f):
    ''' No flow gets left behind'''

    for s, t, u in cartesian_product(V, V, V):
        d = D[int(s-1), int(t-1)]

        if u==s:
            m.addConstr(f.sum(s, t, u, '*')-f.sum(s, t, '*', u)==d, 'conserv')
        elif u==t:
            m.addConstr(f.sum(s, t, u, '*')-f.sum(s, t, '*', u)==-d, 'conserv')
        else:
            m.addConstr(f.sum(s, t, u, '*')-f.sum(s, t, '*', u)==0, 'conserv')

    return

def MinMaxLinkUtil(G,D, verbose = False):
    params = {
    "WLSACCESSID": 'a5796427-505f-41e2-8e78-716fe8f5d2c0',
    "WLSSECRET": 'ebe52f9b-4daa-436a-92c1-1f69a7ef642b',
    "LICENSEID": 2508231,
    "OutputFlag": 0,
    }

    env = gb.Env(params=params)

    # Create instance of optimizer model
    m = gb.Model('netflow', env=env)
    V = np.array([i for i in G.nodes()])

    verboseprint = print
    if not verbose:
        verboseprint = lambda *a: None
        m.setParam('OutputFlag', False )
        m.setParam('LogToConsole', False )

    # Get link costs and capacity
    cost = getCosts(G)
    cap = getCapacity(G)

    arcs, capacity = gb.multidict(cap)

    # Create flow and load variables
    f, l = getLoadandFlowVars(m, V, arcs, cost)

    # Add link utilization constraints
    getLinkUtilization(m, l, f, arcs)

    # Add capacity constraints
    getCapacityConstraint(m, l, capacity, arcs)

    # Add flow conservation constraints
    getFlowConservationConstraint(m, D, V, f)

    # Set objective to max-link utilization (congestion)
    max_cong = m.addVar(name='congestion')
    m.addConstrs(((cost[i,j]*l[i, j])/capacity[i,j]<= max_cong for i, j in arcs))
    m.setObjective(max_cong, gb.GRB.MINIMIZE)

    # Compute optimal solution
    m.optimize()

    # Print solution
    if m.status == gb.GRB.Status.OPTIMAL:
        f_sol = m.getAttr('x', f)
        l_sol = m.getAttr('x', l)
        m_cong = float(max_cong.x)

        verboseprint('\nOptimal traffic flows.')
        verboseprint('f_{i -> j}(s, t) denotes amount of traffic from source'
                     ' s to destination t that goes through link (i, j) in E.')

        for s, t in cartesian_product(V, V):
            for i,j in arcs:
                p = f_sol[s, t, i, j]
                if p > 0:
                    verboseprint('f_{%s -> %s}(%s, %s): %g bytes.'
                                  % (i, j, s, t, p))

        verboseprint('\nTotal traffic through link.')
        verboseprint('l(i, j) denotes the total amount of traffic that passes'
                     ' through edge (i, j).'
        )

        for i, j in arcs:
            p = l_sol[i, j]
            if p > 0:
                verboseprint('%s -> %s: %g bytes.' % (i, j, p))

        verboseprint('\nMaximum weighted link utilization (or congestion):',
                     format(m_cong, '.4f')
        )


        return m_cong
    else:
        return


'''
    Functions for data processing
'''

def create_dataset(dataset, window_size):
    dataX, dataY = [], []

    for i in range(len(dataset)-window_size):
        a = dataset[i:i+window_size, :]
        dataX.append(a)
        dataY.append(dataset[i + window_size, :])

    return np.array(dataX), np.array(dataY)

def train_test_split(dataset, train_ratio):

    train_size = int(len(dataset) * train_ratio)

    train_data = dataset[0:train_size,:]
    test_data = dataset[train_size:len(dataset),:]

    return train_data, test_data


def normalize_matrix(scaler, dataset):

    dataset_norm = np.zeros(dataset.shape)

    for i in range(len(dataset)):
        row = np.reshape(dataset[i, :], (dataset.shape[1],1))
        row = scaler.fit_transform(row)

        dataset_norm[i, :] = np.reshape(row, (dataset.shape[1],))

    return dataset_norm



### Everything Else

In [6]:
'''
    Read one week of data
'''
week = 1

# Format the week with leading zeros
number = f"{week:02d}"

# Path of file
#base_path = f"data/X{number}/X{number}"
base_path = f"X{number}"

# Load file
data = np.loadtxt(base_path)

# Keep first 144 entries from each line
data = data[:, :144]

In [7]:
# Normalize the data
scaler = MinMaxScaler()
data_norm = normalize_matrix(scaler, data)

# Train-Test Split
train_data, test_data = train_test_split(data_norm, 0.8)

# Window the dataset
trainX, trainY = create_dataset(train_data, 10)
testX, testY = create_dataset(test_data, 10)

In [8]:
# Specify model parameters
input_size = trainX.shape[2] # Number of features in input
hidden_size = 200  # Number of features in hidden state
output_size = 144  # Number of output classes
learn_rate = 0.001
epochs = 100
num_layers = 1
batch_size = 32
shuffle = False #don't want to lose the time dependency
num_workers = 2  # Number of subprocesses to use for data loading

In [9]:
# Specify the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [36]:
# Pre-compute MLUs for ground truth training set to avoid for loop lag
G = abilene_topo_lossfn()
baseline_mlu = np.empty((trainY.shape[0]))

for i in range(trainY.shape[0]):
    target = trainY[i, :].reshape((12, 12))
    np.fill_diagonal(target, 0)
    u = MinMaxLinkUtil(G, target)
    baseline_mlu[i] = u

In [39]:
# create torch datasets and torch dataloader

train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(trainX),
                                                 torch.Tensor(trainY),
                                               torch.FloatTensor(baseline_mlu))


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                             num_workers=num_workers,
                                             shuffle=shuffle)

test_dataset  = torch.utils.data.TensorDataset(torch.FloatTensor(testX),
                                                 torch.Tensor(testY))


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                             num_workers=num_workers,
                                             shuffle=shuffle)


In [49]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()

        self.rnn = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )
        self.sigmoid = nn.Sigmoid()
        self.out = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, _ = self.rnn(x, None)  # None represents zero initial hidden state
        out = self.out(r_out[:, -1, :])  # return the last value
        out = self.sigmoid(out)
        return out

import time

# Define training function
def train(model, train_loader, epochs, criterion, optimizer):
    ''' Train ML Model'''

    print_interval = 5
    track_losses = np.zeros(epochs)
    start = time.time()

    for epoch in range(epochs):
        for inputs, targets, mlus in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            mlus = mlus.to(device)

            # Pass data to Model
            optimizer.zero_grad()
            outputs = model(inputs) #size ->: batch_size x output_size

            # Compute the loss
            loss = criterion(outputs, mlus) #size ->: float

            # Compute the gradient and update the network parameters
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        training_loss = loss.item()
        track_losses[epoch] = training_loss

        if (epoch) % (print_interval-1) == 0:
            print('epoch: %4d training loss:%10.3e time:%7.1f'%(epoch, training_loss, time.time()-start))

    return loss

The naive solution uses the output of the MLU and the ground truth MLU, computes the MLU, and performs back propogation

In [50]:
class NaiveLoss(nn.Module):

    def __init__(self, batch_size, G, device):
        super(NaiveLoss, self).__init__()
        self.batch_size = batch_size
        self.G = G
        self.device = device

    def forward(self, inputs, targets):
        # targets in this sense are the baseline mlus
        inputs = inputs.reshape((-1, 12, 12))

        # Use list comprehension to compute u_pred
        u_pred_list = [MinMaxLinkUtil(self.G, inputs[i, :, :].fill_diagonal_(0)) for i in range(self.batch_size)]

        # Convert list to tensor
        u_pred = torch.tensor(u_pred_list).reshape(self.batch_size)

        u_pred.requires_grad_()
        targets.requires_grad_()
        comp_loss = nn.MSELoss()

        # compute the loss
        return comp_loss(targets, u_pred.to(self.device))

In [ ]:
# Create Model
model = RNN(input_size, hidden_size, num_layers)

# Move the model to the specified device
model = model.to(device)

# Create optimizer
optimizer = optim.Adam(model.parameters(), lr=learn_rate)

# Create loss function
G = abilene_topo_lossfn()
criterion = NaiveLoss(batch_size, G, device)

loss = train(model, train_loader, epochs, criterion, optimizer)